In [2]:
%pip install nibabel numpy matplotlib opencv-contrib-python scikit-image

In [3]:
from __future__ import print_function
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import random

https://stackoverflow.com/questions/61833375/loading-a-nifti-through-nibabel-and-using-the-shape-function

In [4]:
def get_slice(nii_data, slice):
  return nii_data[:,:,slice]

In [5]:
def open_nii(path):
  my_img  = nib.load(path)
  nii_data = my_img.get_fdata()
  return nii_data

In [6]:
def show_slice(nii_data, slice):
  plt.imshow(get_slice(nii_data, slice))
  plt.show()

In [7]:
def show(img):
  plt.imshow(img)
  plt.show()

In [8]:
def show_all_slices(nii_data):
  for slice in range(nii_data.shape[2]):
    plt.imshow(get_slice(nii_data,slice))
    plt.show()

In [9]:
def show_with_label(img, label):
  plt.figure(figsize = (50,7))
  plt.subplot(1,2,1)
  plt.imshow(img, 'gray', interpolation='none')
  plt.imshow(label, 'jet', interpolation='none', alpha=0.1)
  plt.show()

In [10]:
def cv_convert(img):
  return cv.normalize(img, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

In [11]:
def is_point_in_mask(mask, x, y):
    # Check if the point is within the mask boundaries
    if 0 <= x < mask.shape[1] and 0 <= y < mask.shape[0]:
        # Check if the pixel value at the specified coordinates is non-zero
        return mask[y, x] == 2 # 0 is background, 1 is pancreas, 2 is cancer
    else:
        # Point is outside the mask boundaries
        return False


In [19]:
def preprocess(ctr):
  img_dir = "../data/Task07_Pancreas/imagesTr"
  files = os.listdir(img_dir)
  label_dir = "../data/Task07_Pancreas/labelsTr"

  for file in files:
    ctr = ctr -1
    num = file.split(".")
    n = num[0].split("_")
    if len(n) < 2:
      print(n)
      continue
    img_path = os.path.join(img_dir, f"pancreas_{n[1]}.nii.gz")
    label_path = os.path.join(label_dir, f"pancreas_{n[1]}.nii.gz")
    # check if there is cancer in label
    img_nii = open_nii(img_path)
    label_nii = open_nii(label_path)
    
    tumor_slices = []
    
    for i in range(0, label_nii.shape[2]-1):
      label_slice = get_slice(label_nii, i)
      if any(2 in nested_list for nested_list in label_slice):
        tumor_slices.append(i)

    if len(tumor_slices) == 0:
      continue

    chosen = random.choice(tumor_slices)
    # loop through slices that have cancer in label

    img = cv_convert(get_slice(img_nii, chosen))
    label = get_slice(label_nii, chosen)
    top = 10000
    left = 10000
    right = 0
    bottom = 0

    for i, row in enumerate(label):
      for j, pixel in enumerate(row):
        if pixel == 0:
          continue
        if i < top:
          top = i
        if j < left:
          left = j
        if i > bottom:
          bottom = i
        if j > right:
          right = j
    
    cv.imwrite(f"../data/preprocessed/images/{n[1]}_{chosen}.png", img[top-1:bottom+2, left-1:right+2])
    cv.imwrite(f"../data/preprocessed/labels/{n[1]}_{chosen}.png", label[top-1:bottom+2, left-1:right+2])
    
    if ctr == 0:
      break


In [20]:
preprocess(500)

['']
